# Digital Assistant - AI Server

This notebook runs the LLM backend for your Desktop Digital Assistant.

**Steps:**
1. Run all cells (Runtime → Run all)
2. Copy the `trycloudflare.com` URL printed at the bottom
3. Paste it into your Desktop App settings

**Model:** Qwen2.5-14B-Instruct

**Tunnel:** Cloudflare (free, no account needed)

**Requirements:** Colab Pro with GPU (A100)

In [ ]:
# Cell 1: Install dependencies
!pip install -q transformers accelerate torch fastapi uvicorn sentencepiece nest-asyncio
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
print("✅ Dependencies installed")

In [ ]:
# Cell 2: Load Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_ID = "Qwen/Qwen2.5-14B-Instruct"

print(f"Loading {MODEL_ID}...")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

print("✅ Model loaded successfully!")

In [ ]:
# Cell 3: Define the AI Chat Engine

SYSTEM_PROMPT = """你是一個桌面數位助理，運行在使用者的 Windows 電腦上。
你可以幫助使用者完成各種任務。

可用技能:
{skills}

重要規則:
1. 永遠用繁體中文回答
2. 回覆必須是合法 JSON 格式
3. 如果需要執行技能，回傳:
   {{"text": "說明文字", "skill": "技能名稱", "args": {{參數}}}}
4. 如果只是聊天，回傳:
   {{"text": "回答內容", "skill": "", "args": {{}}}}
5. 回答要簡潔（除非使用者要求詳細）
6. 當使用者要求建立文件（PPT/Word/Excel），你必須自己生成完整、專業、詳細的內容。絕對不要反問使用者要什麼內容，直接根據主題產生。
7. 建立PPT時，根據主題選擇最適合的 theme:
   - dark: 科技、AI、程式、未來相關
   - corporate: 商業報告、公司簡報、財務
   - nature: 環保、生態、農業、健康
   - warm: 教育、文化、歷史、藝術
   - ocean: 海洋、旅遊、地理、運動
   - minimal: 簡約、設計、建築
8. 建立Word時，使用 # ## ### 標記標題層級，用 - 開頭標記要點。生成結構完整的專業文件。
9. PPT 至少要生成 5 張投影片，每張內容要有 3-5 個要點。
10. **搜尋並製作簡報工作流程（非常重要）**：
    - **關鍵區別**：
      * `search_web`: 只是打開瀏覽器搜尋頁面，不會返回內容給 AI，無法用於製作簡報
      * `fetch_news`: 真正搜尋並返回內容摘要，可以用於製作簡報
    - 當使用者要求「搜尋XX新聞然後做成簡報」、「找XX資料做成簡報」等需求時：
      * **必須使用 fetch_news**，絕對不要使用 search_web
      * 先執行 fetch_news 技能搜尋資料（根據需求判斷 max_results，通常 5-10）
      * 等待搜尋結果返回後，**自動分析使用者原始需求**，判斷是否需要製作簡報
      * 如果使用者要求製作簡報/文件，**在下一輪回應中自動執行 create_ppt/create_docx**
      * 不要等待使用者再次確認，直接根據原始需求完成整個工作流程
      * 簡報內容必須基於搜尋結果，不要編造資料
      * 每張投影片要引用資料來源（如果 fetch_news 有提供來源連結）
    - 範例：使用者說「幫我搜尋最新的AI新聞然後做成簡報」
      * 第一輪：執行 fetch_news("最新的AI新聞", 8)
      * 第二輪（自動）：看到搜尋結果後，立即執行 create_ppt，根據搜尋結果製作簡報
      * 不要只回覆「已搜尋」，要完成整個工作流程

範例:
- 使用者: "幫我做一份關於AI的簡報"
  回覆: {{"text": "好的，已為你建立AI簡報", "skill": "create_ppt", "args": {{"title": "人工智慧簡介", "theme": "dark", "slides_json": [{{"title": "什麼是AI", "content": "人工智慧是模擬人類智慧的技術\\n機器學習讓電腦從資料中自動學習\\n深度學習模仿人腦神經網路結構\\n自然語言處理讓機器理解人類語言"}}, {{"title": "AI的應用", "content": "醫療: AI輔助診斷與藥物開發\\n金融: 風險評估與詐欺偵測\\n教育: 個人化學習與智能tutoring\\n交通: 自動駕駛與路線優化"}}]}}}}
- 使用者: "幫我寫一份環保報告"
  回覆: {{"text": "好的，已建立環保報告", "skill": "create_docx", "args": {{"title": "環境保護報告", "content": "# 前言\\n本報告探討當前環境問題與解決方案。\\n\\n## 現況分析\\n- 全球平均溫度持續上升\\n- 極端氣候事件頻率增加\\n..."}}}}
- 使用者: "幫我搜尋最新的AI新聞然後做成簡報"
  回覆: {{"text": "正在搜尋最新的AI新聞...", "skill": "fetch_news", "args": {{"query": "最新的AI新聞", "max_results": 8}}}}
  （注意：這需要兩步驟，先 fetch_news 取得資料，然後在下一輪對話中根據搜尋結果執行 create_ppt）
"""

def generate_response(messages, skills=None):
    """Generate a response from the model."""
    skills_text = ""
    if skills:
        skills_text = "\n".join(
            f"- {s['name']}: {s['description']} (params: {s.get('params', {})})"
            for s in skills
        )

    system_msg = SYSTEM_PROMPT.format(skills=skills_text or "(無可用技能)")

    conversation = [{"role": "system", "content": system_msg}]
    for msg in messages[-20:]:
        conversation.append({
            "role": msg.get("role", "user"),
            "content": msg.get("content", "")
        })

    text = tokenizer.apply_chat_template(
        conversation,
        tokenize=False,
        add_generation_prompt=True
    )
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=2048,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1,
            do_sample=True
        )

    new_tokens = outputs[0][inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(new_tokens, skip_special_tokens=True)

    return response.strip()

# Quick test
test = generate_response([{"role": "user", "content": "你好"}])
print(f"Test response: {test}")

In [ ]:
# Cell 4: FastAPI Server + Cloudflare Tunnel (free, no account needed)
import json
import re
import subprocess
import threading
import time
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

app = FastAPI(title="Digital Assistant AI Server")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/health")
async def health():
    return {
        "status": "ok",
        "model": "Qwen2.5-14B-Instruct",
        "gpu": torch.cuda.get_device_name(0),
    }

def parse_ai_response(raw_response):
    """Parse the AI response, handling nested JSON with skill/args."""
    # Try parsing the entire response as JSON
    try:
        parsed = json.loads(raw_response)
        if isinstance(parsed, dict) and "text" in parsed:
            return parsed
    except json.JSONDecodeError:
        pass

    # Try to find JSON by matching braces (supports nested objects)
    depth = 0
    start = -1
    for i, ch in enumerate(raw_response):
        if ch == '{':
            if depth == 0:
                start = i
            depth += 1
        elif ch == '}':
            depth -= 1
            if depth == 0 and start >= 0:
                candidate = raw_response[start:i+1]
                try:
                    parsed = json.loads(candidate)
                    if isinstance(parsed, dict) and "text" in parsed:
                        return parsed
                except json.JSONDecodeError:
                    continue

    # Fallback: return as plain text
    return {"text": raw_response, "skill": "", "args": {}}

@app.post("/chat")
async def chat(request: Request):
    body = await request.json()
    messages = body.get("messages", [])
    skills = body.get("skills", [])

    raw_response = generate_response(messages, skills)
    return parse_ai_response(raw_response)

# --- Start Cloudflare Tunnel (free, no signup) ---
def start_cloudflared():
    process = subprocess.Popen(
        ["cloudflared", "tunnel", "--url", "http://localhost:8000"],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    for line in process.stderr:
        if "trycloudflare.com" in line:
            import re as _re
            match = _re.search(r'(https://[^\s]+trycloudflare\.com)', line)
            if match:
                print("=" * 60)
                print("AI Server is running!")
                print("")
                print("Copy this URL to your Desktop App settings:")
                print("")
                print(f"   {match.group(1)}")
                print("")
                print(f"Model: Qwen2.5-14B-Instruct")
                print(f"GPU: {torch.cuda.get_device_name(0)}")
                print("=" * 60)

tunnel_thread = threading.Thread(target=start_cloudflared, daemon=True)
tunnel_thread.start()

time.sleep(2)
print("Starting server... tunnel URL will appear shortly:")

import asyncio
config = uvicorn.Config(app, host="0.0.0.0", port=8000)
server = uvicorn.Server(config)
loop = asyncio.get_event_loop()
loop.run_until_complete(server.serve())